#  Webscraping avec python
# **Python**
# **TP 6 - Scraping avec BeautifulSoup**

**Inspiré de : [Cours de Python pour le Data Scientist - Lino Galiana](https://linogaliana-teaching.netlify.app/manipulation/webscraping/)**


***

[**Notions**](#notions)

1. [Balises HTML](#1)
2. [Parent et enfant](#2)
3. [Utiliser BeautifulSoup](#3)


*** 

[**Exercices**](#exercices)

- Exercice 1
- Exercice 2

***


## Exercices

## Exercice 1 : obtenir la liste des équipes de Ligue 1

Sur la page https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_2019-2020, on veut scraper le tableau de la section Participants (celui avec ces colonnes : (Club, Dernière montée, Budget en M€, ...)

#### 1) Trouver le tableau


In [10]:
!pip3 show lxml
#pour l'installer :  !pip3 install lxml

Name: lxml
Version: 4.6.1
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD
Location: /usr/local/lib/python3.9/site-packages
Requires: 
Required-by: fraddress, python-docx, svglib, yfinance


In [11]:
import requests
from bs4 import BeautifulSoup

url_ligue_1 = "https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_2019-2020"

# Requête et son contenu
response = requests.get(url_ligue_1)
content = response.content

# On parse le html
page = BeautifulSoup(content, 'html')

# On trouve le tableau
tableau_participants = page.find(
    'table', 
    class_=['wikitable', 'sortable'] # ou directement 'wikitable sortable'
)

# Dans le tableau, on trouve le fuls tbody
table_body = tableau_participants.find('tbody')

print(type(table_body))

<class 'bs4.element.Tag'>


In [12]:
from urllib import request
import ssl


# Problème pour lire depuis https avec read_html
# https://stackoverflow.com/questions/50969583/pandas-raises-ssl-certificateerror-when-using-method-read-html-for-https-resourc

url = url_ligue_1 #"https://example.com/data.html"
context = ssl._create_unverified_context()
response = request.urlopen(url, context=context)
html = response.read()

import pandas as pd
l_df = pd.read_html(html)
print(len(l_df), "tables récupérées et parsées")

34 tables récupérées et parsées


In [17]:
df[17]

,Unnamed: 0,Passeur,Club,Pas.
0,1,Ángel Di María,Paris Saint-Germain,14
1,2,Islam Slimani,AS Monaco,8
2,3,Yoann Court,Stade brestois 29,7
3,4,Pierre Lees-Melou,OGC Nice,6
4,5,Neymar,Paris Saint-Germain,6
5,6,Jonathan Ikoné,LOSC Lille,6
6,7,Romain Del Castillo,Stade rennais FC,5
7,8,Kylian Mbappé,Paris Saint-Germain,5
8,9,Moses Simon,FC Nantes,5
9,10,Gaëtan Laborde,Montpellier Hérault SC,5


#### 2) Récupérer chaque ligne du table


In [2]:
# on recherche toutes les lignes du tableau avec la balise "tr"
rows = table_body.find_all('tr')

In [3]:
# Example : 
row = rows[12]
row = row.text.split("\n")
row = [x for x in row if x != ""]

In [4]:
# On reproduit cette logique sur toutes les lignes
rows_cleaned = []
for row in rows:
    row_cleaned = row.text.split("\n")
    row_cleaned = [x for x in row_cleaned if x != ""]
    rows_cleaned.append(row_cleaned)
    
rows_cleaned = [[x for x in row.text.split("\n") if x != ""] for row in rows] 

In [5]:
# Example
row = rows[12]
values = row.find_all("td")
values = [tag.text.strip("\n") for tag in values]
values

['FC Nantes',
 '2013',
 '70',
 '12e',
 ' Christian Gourcuff',
 '2019',
 'Stade de la Beaujoire - Louis Fonteneau',
 '35\xa0322',
 '51']

In [6]:
# On reproduit cette logique sur toutes les lignes
l_values = []

# On récupère les entêtes avec th
values = rows[0].find_all("th")
values = [tag.text.strip("\n") for tag in values]
l_values.append(values)

for row in rows:
    # On récupère les valeurs dans les cellules
    values = row.find_all("td")
    values = [tag.text.strip("\n") for tag in values]
    l_values.append(values)

In [20]:
# l_values

## Exercice 2
- A partir de la suivante : [pokedex](https://pokemondb.net/pokedex/national), l'objectif est de constituer la liste de tous les pokemons
 - 1. Dans un premier temps, on voudrait la liste avec nom, numéro et types pour chaque pokémon
 - 2. Ensuite, on souhaiterait aussi garder l'url pour chaque pokemon (par exemple en utilisant l'attribut ``href``) pour pouvoir ensuite faire une requête ``GET``sur cette page (par exemple : https://pokemondb.net/pokedex/venusaur) et récupérer les données de taille et de poids

In [18]:
## Question 1 

import requests
from bs4 import BeautifulSoup
import os


url = "https://pokemondb.net/pokedex/national"

# Requête
response = requests.get(url)
content = response.content
page = BeautifulSoup(content, 'html.parser')


# Division infocard pour chaque pokemon
info_cards = page.find_all("div", class_="infocard")

l_pokemons = []

for info_card in info_cards[:15]:
   
    # On filtre sur la classe et on prend l'attribut text
    name_tag = info_card.find('a', class_="ent-name")
    name = name_tag.text
    number = info_card.find('small').text[1:]
    
    url = "https://pokemondb.net/pokedex/" + name_tag.get("href").split("/")[2]
    
    # On peut le trouver ainsi
    # types = info_card.find_all("small")[1]
    #print(types)
    
    # On peut aussi utiliser
    types  = info_card.find('small').find_next("small")#.text
    
    
    #print(types)
    types = [t.strip() for t in types.text.split(" · ")]
    
    d = {"name":name, "number":number, "types":types, "url": url}
   
    l_pokemons.append(d)
## Question 1 (partielle, il n'y a pas les types)


In [20]:
# from pprint import pprint
# pprint(l_pokemons)

In [21]:
l_pokemons[0]

{'name': 'Bulbasaur',
 'number': '0001',
 'types': ['Grass', 'Poison'],
 'url': 'https://pokemondb.net/pokedex/bulbasaur'}

In [22]:
for pokemon in l_pokemons[:2]:
    url = pokemon["url"]
    print(url)
    
    # Requête
    response = requests.get(url)
    content = response.content
    page = BeautifulSoup(content, 'html.parser')
    
    # On récupère les nom des variables (en-têtes mais à gauche dans cette table)
    ths = page.find("table", class_="vitals-table").find_all("th")
    
    # On utilise find_next après avoir trouvé l'entête
    # In this order
    data = [th.find_next().text for th in ths if th.text in ["Height", "Weight"]]
    print(data)
    
    height = data[0].split("(")[0].replace("\xa0", " ").strip()
    weight = data[1].split("(")[0].replace("\xa0", " ").strip()

    pokemon["height"] = height
    pokemon["weight"] = weight

https://pokemondb.net/pokedex/bulbasaur
['0.7\xa0m (2′04″)', '6.9\xa0kg (15.2\xa0lbs)']
https://pokemondb.net/pokedex/ivysaur
['1.0\xa0m (3′03″)', '13.0\xa0kg (28.7\xa0lbs)']
